# CSE5522 LAB3

In [307]:
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np

Load all csv file

In [308]:
Observation = 'observationProbs.csv'
Transition = 'transitionProbs.csv'
Testdata = 'testData.csv'


obser_df = pd.read_csv(Observation)
trans_df = pd.read_csv(Transition)
test_df = pd.read_csv(Testdata)

In [309]:
obser_df

,P(x|...),C,H
0,1,0.6407,0.0002
1,2,0.1481,0.5341
2,3,0.2122,0.4657


In [310]:
trans_df

,P(x|...),C,H,START
0,C,0.86,0.07,0.5
1,H,0.07,0.86,0.5
2,STOP,0.07,0.07,0.0


In [311]:
test_df

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,1,2,3,3,2,3
1,2,2,3,2,2,0
2,3,3,1,3,3,1
3,4,2,1,1,0,0
4,5,1,1,1,2,3
5,6,1,3,1,1,0
6,7,3,2,3,0,0
7,8,2,2,1,1,0
8,9,1,3,2,3,3
9,10,1,1,1,0,0


# part1

In [337]:
hidden_var = np.array(trans_df)[:2,0]
hidden_var

array(['C', 'H'], dtype=object)

In [338]:
observation = np.array(obser_df)[:,0]
observation

array([1., 2., 3.])

In [339]:
test = np.array(test_df)[:,1:6]
test

array([[2, 3, 3, 2, 3],
       [2, 3, 2, 2, 0],
       [3, 1, 3, 3, 1],
       [2, 1, 1, 0, 0],
       [1, 1, 1, 2, 3],
       [1, 3, 1, 1, 0],
       [3, 2, 3, 0, 0],
       [2, 2, 1, 1, 0],
       [1, 3, 2, 3, 3],
       [1, 1, 1, 0, 0]], dtype=int64)

Get start probility, stop probility, transition probility and emission probility from tables that professor provided 

In [342]:
start_pro={}
stop_pro={}
trans_pro={}
emis_pro={}
start_pro = {'C':0.5, 'H':0.5}
stop_pro = {'C': 0.07, 'H':0.07}
trans_pro={
    'C':{'C':0.86,'H':0.07},
    'H':{'C':0.07,'H':0.86}
}
for n in range(len(hidden_var)):
    emis_temp = {}
    emis_pro[hidden_var[n]] = emis_temp
    for m in range(len(observation)):
        emis_temp[observation[m]]=np.array(obser_df)[:,n+1][m]

print(start_pro, stop_pro, trans_pro, emis_pro)


{'C': 0.5, 'H': 0.5} {'C': 0.07, 'H': 0.07} {'C': {'C': 0.86, 'H': 0.07}, 'H': {'C': 0.07, 'H': 0.86}} {'C': {1.0: 0.6407, 2.0: 0.1481, 3.0: 0.2122}, 'H': {1.0: 0.0002, 2.0: 0.5341, 3.0: 0.4657}}


In [343]:
def catch_value(probility, value):
    for val, pro in probility.items():
        if value == pro:
            return val

viterbi algorithm function

In [ ]:
def viterbi(observation,hidden_var,start_pro,stop_pro,trans_pro,emis_pro):
    V=[{}]
    path={}
    for i in hidden_var:
        V[0][i] = start_pro[i] * emis_pro[i][observation[0]]
        path[i] = [i]
    for n in range(1,len(observation)):
        V.append({})
        newpath = {}
        for m in hidden_var:
            pro={}
            for x in hidden_var:
                v_pro = V[n-1][x]
                transition = trans_pro[x][m]
                emission = emis_pro[m][observation[n]]
                pro[x]=v_pro*transition*emission
            temp_pro = max(pro.values())
            var = catch_value(pro,temp_pro)
            V[n][m]=temp_pro
            newpath[m]= path[var]+[m]
        path = newpath
    new_pro={}
    for y in hidden_var:
        new_pro[y]=V[len(observation)-1][y]*stop_pro[x]
    temp_pro = max(new_pro.values())
    var = catch_value(new_pro,temp_pro)
    return path[var]

print result solution:

In [345]:

test = np.array( pd.read_csv(Testdata))[:,1:6]
test
for i in range(len(test)):
    observation  = test[i][test[i]!=0]
    print(observation)
    print(viterbi(observation,hidden_var,start_pro,stop_pro,trans_pro,emis_pro))

[2 3 3 2 3]
['H', 'H', 'H', 'H', 'H']
[2 3 2 2]
['H', 'H', 'H', 'H']
[3 1 3 3 1]
['C', 'C', 'C', 'C', 'C']
[2 1 1]
['C', 'C', 'C']
[1 1 1 2 3]
['C', 'C', 'C', 'C', 'C']
[1 3 1 1]
['C', 'C', 'C', 'C']
[3 2 3]
['H', 'H', 'H']
[2 2 1 1]
['H', 'H', 'C', 'C']
[1 3 2 3 3]
['C', 'H', 'H', 'H', 'H']
[1 1 1]
['C', 'C', 'C']


Conclusion: Based on transition table and emission table, I can predict wether each dau is hot or cold by using viterbi.

# part2

In [319]:

def weights(hidden_var,observation, start_pro, trans_pro, emis_pro,time):
    pro = random.random()
    example = {}
    state = []
    new_trans_pro = normalize(hidden_var, trans_pro)
    for i in range(time):
        s=""
        if pro > 0.5 :
            s = "H"
        else:
            s = "C"
        state.append(s)
        weight = emis_pro[s][observation[0]]
        for n in range(1, len(observation)):
            trans = new_trans_pro["C"][s]
            pro = random.random()
            if pro> trans:
                s = "H"
            else:
                s="C"
            state.append(s)
            weight = weight * emis_pro[s][observation[n]]
        if tuple(state) in example:
            example[tuple(state)]=example[tuple(state)]+weight
        else:
            example[tuple(state)] = weight
        state=[]
    max_w = max(example.values())
    max_s = catch_value(example,max_w)
    return max_s
            

normalize the transition probility

In [320]:
def normalize(hidden_var, trans_pro):
    for x in hidden_var:
        pro = 0
        for y in hidden_var:
            pro = pro+trans_pro[y][x]
        for element in hidden_var:
            trans_pro[element][x]=trans_pro[element][x]/pro
    return trans_pro

print all solution:

In [321]:
for i in range(len(test)):
    observation  = test[i][test[i]!=0]
    print(observation)
    print("We take 50000 sample to get solution:", weights(hidden_var,observation,start_pro,trans_pro,emis_pro,50000))

[2 3 3 2 3]
We take 50000 sample to get solution: ('H', 'H', 'H', 'H', 'H')
[2 3 2 2]
We take 50000 sample to get solution: ('H', 'H', 'H', 'H')
[3 1 3 3 1]
We take 50000 sample to get solution: ('C', 'C', 'C', 'C', 'C')
[2 1 1]
We take 50000 sample to get solution: ('C', 'C', 'C')
[1 1 1 2 3]
We take 50000 sample to get solution: ('C', 'C', 'C', 'C', 'C')
[1 3 1 1]
We take 50000 sample to get solution: ('C', 'C', 'C', 'C')
[3 2 3]
We take 50000 sample to get solution: ('H', 'H', 'H')
[2 2 1 1]
We take 50000 sample to get solution: ('H', 'H', 'C', 'C')
[1 3 2 3 3]
We take 50000 sample to get solution: ('C', 'H', 'H', 'H', 'H')
[1 1 1]
We take 50000 sample to get solution: ('C', 'C', 'C')


I use 50000 samples to test and the solutions of likelihood can match the result of viterbi. In fact, the results exited some wrong values. However, normalize data can solve this problem.

# Bonus 

Forward-Backward

In [347]:
def forward(observation, hidden_var, star_pro, trans_pro, emmis_pro,state):
    forward
    